# Data analysis and preparation 

### Lab №2

#### Setup

##### Run the next cell to import and configure the Python libraries.

In [1]:
import os
import pandas as pd
import urllib.request
from datetime import datetime
import numpy as np
import shutil
print("Setup Completed")

Setup Completed


##### Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс(номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження. Передбачити повторні запуски скрипту, довантаження нових даних та колізію даних;

In [2]:
def download_vhi_data():
    data_directory = 'VHI'
    
    shutil.rmtree(data_directory, ignore_errors=True)
    os.makedirs(data_directory, exist_ok=True)
    
    for region_id in range(1, 28):
        url = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={region_id}&year1=1982&year2=2023&type=Mean'
        
        try:
            response = urllib.request.urlopen(url)
            data = response.read()
            
            current_time = datetime.now()
            timestamp = current_time.strftime("%Y-%m-%d_%H-%M-%S")
            
            file_name = f'vhi_region_{region_id}_{timestamp}.csv'
            file_path = os.path.join(data_directory, file_name)
            
            with open(file_path, 'wb') as file:
                file.write(data)
            
            print(f"! VHI data for region id {region_id} saved as {file_name}!")
        except Exception as e:
            print(f"!!! Error loading data for region id {region_id}: {str(e)}!!!")
        
download_vhi_data()

! VHI data for region id 1 saved as vhi_region_1_2025-02-18_17-05-33.csv!
! VHI data for region id 2 saved as vhi_region_2_2025-02-18_17-05-34.csv!
! VHI data for region id 3 saved as vhi_region_3_2025-02-18_17-05-35.csv!
! VHI data for region id 4 saved as vhi_region_4_2025-02-18_17-05-36.csv!
! VHI data for region id 5 saved as vhi_region_5_2025-02-18_17-05-36.csv!
! VHI data for region id 6 saved as vhi_region_6_2025-02-18_17-05-37.csv!
! VHI data for region id 7 saved as vhi_region_7_2025-02-18_17-05-38.csv!
! VHI data for region id 8 saved as vhi_region_8_2025-02-18_17-05-39.csv!
! VHI data for region id 9 saved as vhi_region_9_2025-02-18_17-05-40.csv!
! VHI data for region id 10 saved as vhi_region_10_2025-02-18_17-05-41.csv!
! VHI data for region id 11 saved as vhi_region_11_2025-02-18_17-05-41.csv!
! VHI data for region id 12 saved as vhi_region_12_2025-02-18_17-05-42.csv!
! VHI data for region id 13 saved as vhi_region_13_2025-02-18_17-05-43.csv!
! VHI data for region id 14 sa

##### 2. Зчитати завантажені текстові файли у фрейм. Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [3]:
def load_vhi_data(directory_path):
    if not os.path.exists(directory_path):
        print(f"Error: Directory '{directory_path}' not found!")
        return pd.DataFrame()
    
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    files = os.listdir(directory_path)
    result_df = pd.DataFrame()

    for i in range(len(files)):
        filepath = os.path.join(directory_path, files[i])
        df = pd.read_csv(filepath, header=1, names=headers, skiprows=1)  

        df = df.dropna(subset=['VHI'])
        df = df[df['VHI'] != -1]
        df = df.replace({'<tt>': '', '<pre>': ''})
        df['Area_ID'] = i + 1

        if 'empty' in df.columns:
            df.drop('empty', axis=1, inplace=True)
            
        if not df.empty:
            result_df = pd.concat([result_df, df], ignore_index=True)

    return result_df
            
directory = "VHI"
df = load_vhi_data(directory)
print(df.head())
print(df.tail())



    

   Year  Week    SMN     SMT    VCI    TCI    VHI  Area_ID
0  1982   2.0  0.063  261.53  55.89  38.20  47.04        1
1  1982   3.0  0.063  263.45  57.30  32.69  44.99        1
2  1982   4.0  0.061  265.10  53.96  28.62  41.29        1
3  1982   5.0  0.058  266.42  46.87  28.57  37.72        1
4  1982   6.0  0.056  267.47  39.55  30.27  34.91        1
       Year  Week    SMN     SMT    VCI    TCI    VHI  Area_ID
57586  2023  48.0  0.101  277.36  37.50  15.86  26.68       27
57587  2023  49.0  0.097  275.73  38.12  16.77  27.45       27
57588  2023  50.0  0.097  275.42  40.98  13.16  27.07       27
57589  2023  51.0  0.097  274.98  43.98  11.14  27.56       27
57590  2023  52.0  0.098  274.62  47.00  10.39  28.70       27


##### 3. Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA.

In [4]:
area_mapping = {
        1: 'Vinnytsia', 2: 'Volyn', 3: 'Dnipropetrovsk', 4: 'Donetsk',
        5: 'Zhytomyr', 6: 'Zakarpattia', 7: 'Zaporizhzhia', 8: 'Ivano-Frankivsk',
        9: 'Kyivska', 10: 'Kirovohrad', 11: 'Luhansk', 12: 'Lviv',
        13: 'Mykolaiv', 14: 'Odesa', 15: 'Poltava', 16: 'Rivne',
        17: 'Sumy', 18: 'Ternopil', 19: 'Kharkiv', 20: 'Kherson',
        21: 'Khmelnytskyi', 22: 'Cherkasy', 23: 'Chernivtsi', 24: 'Chernihiv',
        25: 'Sevastopol', 26: 'Kyiv', 27: 'Crimea'
    }
df['Area_ID'] = df['Area_ID'].replace(area_mapping)

print(df.head())
print(df.tail())


   Year  Week    SMN     SMT    VCI    TCI    VHI    Area_ID
0  1982   2.0  0.063  261.53  55.89  38.20  47.04  Vinnytsia
1  1982   3.0  0.063  263.45  57.30  32.69  44.99  Vinnytsia
2  1982   4.0  0.061  265.10  53.96  28.62  41.29  Vinnytsia
3  1982   5.0  0.058  266.42  46.87  28.57  37.72  Vinnytsia
4  1982   6.0  0.056  267.47  39.55  30.27  34.91  Vinnytsia
       Year  Week    SMN     SMT    VCI    TCI    VHI Area_ID
57586  2023  48.0  0.101  277.36  37.50  15.86  26.68  Crimea
57587  2023  49.0  0.097  275.73  38.12  16.77  27.45  Crimea
57588  2023  50.0  0.097  275.42  40.98  13.16  27.07  Crimea
57589  2023  51.0  0.097  274.98  43.98  11.14  27.56  Crimea
57590  2023  52.0  0.098  274.62  47.00  10.39  28.70  Crimea


##### 4. Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
##### 4.1. Ряд VHI для області за вказаний рік

In [ ]:
def get_vhi_series(df, region, year):
    filtered_df = df[(df['Area_ID'] == region) & (df['Year'] == year)]
    if filtered_df.empty:
        print(f"No data available for {region} in {year}")
        return None
    return filtered_df[['Year', 'Week', 'VHI', 'Area_ID']]

wanted_reg = input("Region: ")
wanted_year = input("Year: ")
wanted_year = str(wanted_year)

vhi_list = get_vhi_series(df, wanted_reg, wanted_year)

print(f"! VHI series for {wanted_reg} in {wanted_year} year:")
print(vhi_list)


! VHI series for Kyiv in 1999 year:
       Year  Week    VHI Area_ID
54178  1999   1.0  51.08    Kyiv
54179  1999   2.0  51.41    Kyiv
54180  1999   3.0  52.96    Kyiv
54181  1999   4.0  58.13    Kyiv
54182  1999   5.0  61.51    Kyiv
54183  1999   6.0  58.69    Kyiv
54184  1999   7.0  53.71    Kyiv
54185  1999   8.0  48.93    Kyiv
54186  1999   9.0  45.63    Kyiv
54187  1999  10.0  44.32    Kyiv
54188  1999  11.0  43.18    Kyiv
54189  1999  12.0  44.01    Kyiv
54190  1999  13.0  45.74    Kyiv
54191  1999  14.0  46.39    Kyiv
54192  1999  15.0  47.48    Kyiv
54193  1999  16.0  49.82    Kyiv
54194  1999  17.0  54.19    Kyiv
54195  1999  18.0  59.11    Kyiv
54196  1999  19.0  61.17    Kyiv
54197  1999  20.0  62.69    Kyiv
54198  1999  21.0  61.76    Kyiv
54199  1999  22.0  57.49    Kyiv
54200  1999  23.0  53.51    Kyiv
54201  1999  24.0  50.73    Kyiv
54202  1999  25.0  47.37    Kyiv
54203  1999  26.0  43.35    Kyiv
54204  1999  27.0  40.37    Kyiv
54205  1999  28.0  38.58    Kyiv
54206  

##### 4.2 Пошук екстремумів (min та max) для вказаних областей та років,середнього, медіани;

In [ ]:
def analyze_vhi(df, region, year):
    df['Year'] = df['Year'].astype(str)
    filtered_df = df[(df['Area_ID'] == region) & (df['Year'] == year)]
    
    if filtered_df.empty:
        print(f"No data available for {region} in {year}")
        return None
    
    vhi_min = filtered_df['VHI'].min()
    vhi_max = filtered_df['VHI'].max()
    vhi_mean = filtered_df['VHI'].mean()
    vhi_median = filtered_df['VHI'].median()

    return {
        'Min VHI': vhi_min,
        'Max VHI': vhi_max,
        'Mean VHI': vhi_mean,
        'Median VHI': vhi_median
    }

def display_vhi_stats(stats, region, year):
    if stats is None:
        print(f"No data available for {region} in {year}")
        return
    
    print(f"\nVHI Statistics for {region} in {year}:")
    print("-" * 40)
    print(f" Min VHI: {stats['Min VHI']:.2f}")
    print(f" Max VHI: {stats['Max VHI']:.2f}")
    print(f" Mean VHI: {stats['Mean VHI']:.2f}")
    print(f" Median VHI: {stats['Median VHI']:.2f}")
    print("-" * 40)


wanted_reg = input("Region: ")
wanted_year = input("Year: ")
wanted_year = str(wanted_year)
vhi_stats = analyze_vhi(df, wanted_reg, wanted_year)


display_vhi_stats(vhi_stats, wanted_reg, wanted_year)



VHI Statistics for Kyiv in 1999:
----------------------------------------
 Min VHI: 27.63
 Max VHI: 62.69
 Mean VHI: 45.71
 Median VHI: 44.44
----------------------------------------


##### 4.3 Ряд VHI за вказаний діапазон років для вказаних областей

In [11]:
def get_vhi_range(df, regions, start_year, end_year):
    df['Year'] = df['Year'].astype(int)
    filtered_df = df[(df['Area_ID'].isin(regions)) & (df['Year'].between(start_year, end_year))]
    return filtered_df[['Year', 'Week', 'Area_ID', 'VHI']]

wanted_reg = input("Region (comma-separated if multiple): ").split(",") 
wanted_reg = [region.strip() for region in wanted_reg]  

wanted_year_start = int(input("Start Year: ")) 
wanted_end_year = int(input("End Year: "))  
vhi_range = get_vhi_range(df, wanted_reg, wanted_year_start, wanted_end_year)
print(vhi_range)

       Year  Week Area_ID    VHI
28582  1999   1.0   Odesa  46.35
28583  1999   2.0   Odesa  51.87
28584  1999   3.0   Odesa  59.72
28585  1999   4.0   Odesa  63.51
28586  1999   5.0   Odesa  63.63
...     ...   ...     ...    ...
29389  2014  48.0   Odesa  41.98
29390  2014  49.0   Odesa  46.52
29391  2014  50.0   Odesa  47.25
29392  2014  51.0   Odesa  46.89
29393  2014  52.0   Odesa  46.79

[812 rows x 4 columns]


##### 4.4 Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25). Повернути роки, назви областей з екстремальними посухами та значення VHI;

In [14]:
def detect_extreme_droughts_years(df, threshold_percentage=20):
    result = []
    extreme_threshold = 15
    total_regions = df['Area_ID'].nunique()
    affected_threshold = (threshold_percentage / 100) * total_regions
    drought_counts = df[df['VHI'] < extreme_threshold].groupby('Year')['Area_ID'].nunique()
    extreme_drought_years = drought_counts[drought_counts > affected_threshold]

    

    
    for year in extreme_drought_years.index:
        affected_regions = df[(df['Year'] == year) & (df['VHI'] < extreme_threshold)][['Area_ID', 'VHI']]
        result.append((year, affected_regions))
    
    return result

extreme_droughts = detect_extreme_droughts_years(df, threshold_percentage=20)
for year, regions in extreme_droughts:
    print(f"Year {year}: Extreme drought in regions:")
    print(regions)


Year 2000: Extreme drought in regions:
              Area_ID    VHI
3081            Volyn  12.51
3082            Volyn  10.60
3083            Volyn  11.20
3084            Volyn  12.32
3085            Volyn  14.65
5212   Dnipropetrovsk  14.89
5213   Dnipropetrovsk  12.76
5214   Dnipropetrovsk   7.81
5215   Dnipropetrovsk   6.49
5216   Dnipropetrovsk   6.58
5217   Dnipropetrovsk   6.71
5218   Dnipropetrovsk   7.56
5219   Dnipropetrovsk   9.25
5220   Dnipropetrovsk  10.94
5221   Dnipropetrovsk  12.28
22278         Luhansk  14.64
22279         Luhansk  11.82
22280         Luhansk  10.81
22281         Luhansk  10.68
22282         Luhansk  12.30
22283         Luhansk  14.24
24412            Lviv  13.14
24413            Lviv   9.50
24414            Lviv   8.14
24415            Lviv   9.69
24416            Lviv  11.20
24417            Lviv  11.36
24418            Lviv  12.77
32944           Rivne  12.26
32945           Rivne  11.28
32946           Rivne  11.25
32947           Rivne  11.38
3294